# 1. Initializations

## 1.1 General imports

In [ ]:
### general

### data management
import pandas as pd
import numpy as np

### machine learning (scikit-learn)
from scipy.sparse import csr_matrix
import sklearn.metrics.pairwise as dist
from sklearn.decomposition import TruncatedSVD


### graphical
import matplotlib.pyplot as plt
# for jupyter notebook management
%matplotlib inline
import seaborn as sns


## 1.2 General dataframe functions

In [ ]:
import smartcheck.dataframe_common as dfc

## 1.3 General classification functions

In [ ]:
# None

# 2. Loading and Data Quality

## 2.1 Loading of data sets and general exploration

In [ ]:
df_rating_raw = dfc.load_dataset_from_config('rating_data', sep=',')

if df_rating_raw is not None and isinstance(df_rating_raw, pd.DataFrame):
    dfc.log_general_info(df_rating_raw)
    nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_rating_raw)
    if nb_first != nb_total:
        print(dfc.duplicates_index_map(df_rating_raw))
    df_rating = df_rating_raw.copy()
    display(df_rating.head())

In [ ]:
df_rating_desc = df_rating.select_dtypes(include=np.number).describe()
display(df_rating_desc)
df_rating_cr = df_rating.select_dtypes(include=np.number).corr()
display(df_rating_cr)

In [ ]:
df_book_raw = dfc.load_dataset_from_config('book_data', sep=',')

if df_book_raw is not None and isinstance(df_book_raw, pd.DataFrame):
    dfc.log_general_info(df_book_raw)
    nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_book_raw)
    if nb_first != nb_total:
        print(dfc.duplicates_index_map(df_book_raw))
    df_book = df_book_raw.copy()
    display(df_book.head())

In [ ]:
df_book_desc = df_book.select_dtypes(include=np.number).describe()
display(df_book_desc)
df_book_cr = df_book.select_dtypes(include=np.number).corr()
display(df_book_cr)

In [ ]:
# jointure
df_gr = df_rating.merge(df_book, on='book_id',how='left')
df_gr = df_gr[['user_id','title','rating']]
print(df_gr.info())

In [ ]:
# filtrage pour les performances sur les utilisateurs et titres les plus fréquents
frequent_users = df_gr.user_id.value_counts()
frequent_users = frequent_users[frequent_users>100].index
frequent_titles = df_gr.title.value_counts()
frequent_titles = frequent_titles[frequent_titles>1000].index
df_gr = df_gr[(df_gr.user_id.isin(frequent_users))&(df_gr.title.isin(frequent_titles))]
print(df_gr.shape)
print(df_gr.user_id.nunique())
print(df_gr.title.nunique())

# 3. Data Viz

In [ ]:
livre_stats = df_gr.groupby('title')['rating'].agg(['count', 'mean']).reset_index()
# f)
C = livre_stats['count'].mean()
M = livre_stats['mean'].mean()
# On définit la fonction 'bayesian_avg' qui calcule la note bayésienne pour chaque livre en utilisant les valeurs de C et M calculées précédemment.
def bayesian_avg(df_gr):
    return (C * M + df_gr.sum()) / (C + df_gr.count())
# On calcule la note bayésienne pour chaque livre en utilisant la fonction 'bayesian_avg'.
bayesian_avg_ratings = df_gr.groupby('title')['rating'].agg(bayesian_avg).reset_index()
# On renomme les colonnes du DataFrame 'bayesian_avg_ratings' pour les rendre plus explicites.
bayesian_avg_ratings.columns = ['title', 'bayesian_avg']
# On fusionne 'livre_stats' avec les moyennes bayésiennes en utilisant le titre comme clé et on tri par moyenne bayesienne en ordre décroissant.
book_stats = livre_stats.merge(bayesian_avg_ratings, on='title').sort_values('bayesian_avg', ascending=False)
# Sélection des 10 premiers livres les mieux notés
best_rated_books = book_stats[['title', 'bayesian_avg']].head(10)
# Affichage du graphique
sns.barplot(y='title', x='bayesian_avg', data=best_rated_books, orient = 'h')
plt.title(f'Top 10 Livres les mieux notés : moyenne bayesienne')
plt.xlabel("Note moyenne")

# 4. Table Pivot

In [ ]:
mat_ratings = df_gr.pivot_table(columns='title', index='user_id', values='rating')
display(mat_ratings.head(10))
print(mat_ratings.shape)
# Conservation des index de la matrice pleine
user_mapping = mat_ratings.index
title_mapping = mat_ratings.columns
# Conversion en matrice creuse (Compressed Sparse Row format)
mat_ratings = mat_ratings.fillna(0)
mat_sparse_ratings = csr_matrix(mat_ratings.values)
user_ratings = mat_sparse_ratings
title_ratings = mat_sparse_ratings.T

# 5. Matrices de similarités (utilisateurs ou item)

In [ ]:
# calcul de la similarité utilisateur et affichage de la grille des 30 premiers
user_similarity = pd.DataFrame(
    dist.cosine_similarity(user_ratings),
    index=user_mapping, 
    columns=user_mapping
)
display(user_similarity)
sns.heatmap(user_similarity.iloc[:30,:30], center=0)

In [ ]:
# calcul de la similarité utilisateur et affichage de la grille des 30 premiers
title_similarity = pd.DataFrame(
    dist.cosine_similarity(title_ratings),
    index=title_mapping, 
    columns=title_mapping
)
display(title_similarity)
sns.heatmap(title_similarity.iloc[:30,:30], center=0)

# 6. Prédictions des K plus proches voisins

In [ ]:
def pred_user(mat_ratings, user_similarity, k, user_id):

    # Sélectionner dans mat_ratings les livres qui n'ont pas été encore lu par le user
    to_predict = mat_ratings.loc[user_id][mat_ratings.loc[user_id]==0]
    # Sélectionner les k users les plus similaires en excluant le user lui-même
    similar_users = user_similarity.loc[user_id].sort_values(ascending=False)[1:k+1]
    # Calcul du dénominateur
    norm = np.sum(np.abs(similar_users))
    for i in to_predict.index:
        # Récupérer les notes des users similaires associées au film i
        ratings = mat_ratings[i].loc[similar_users.index]        
        # Calculer le produit scalaire entre ratings et similar_users
        scalar_prod = np.dot(ratings, similar_users)        
        # Calculer la note prédite pour le titre i
        pred = scalar_prod / norm
        # Remplacer par la prédiction
        to_predict[i] = pred
        
    return to_predict

In [ ]:
# Application
userId = 7
user_preferences = df_gr[(df_gr['user_id']==userId) & (df_gr['rating']>=4)]
user_preferences.sort_values('rating', ascending=False).drop_duplicates().head(10)
display(user_preferences.head(10))
reco_user = pred_user(mat_ratings, user_similarity, 3, userId)
print(reco_user.sort_values(ascending=False).head(10))


In [ ]:
def pred_title(mat_ratings, item_similarity, k, user_id):

    # Sélectionner dans mat_ratings les livres qui n'ont pas été encore lu par le user
    to_predict = mat_ratings.loc[user_id][mat_ratings.loc[user_id]==0]
    # Itérer sur tous ces livres 
    for i in to_predict.index:
        #Trouver les k livres les plus similaires en excluant le livre lui-même
        similar_items = item_similarity.loc[i].sort_values(ascending=False)[1:k+1]
        # Calcul de la norme du vecteur similar_items
        norm = np.sum(np.abs(similar_items))
        # Récupérer les notes données par l'utilisateur aux k plus proches voisins
        ratings = mat_ratings.loc[user_id][similar_items.index]
        # Calculer le produit scalaire entre ratings et similar_items
        scalar_prod = np.dot(ratings,similar_items)  
        #Calculer la note prédite pour le titre i
        pred = scalar_prod / norm
        # Remplacer par la prédiction
        to_predict[i] = pred

    return to_predict


In [ ]:
# Application
userId = 7
user_preferences = df_gr[(df_gr['user_id']==userId) & (df_gr['rating']>=4)]
user_preferences.sort_values('rating', ascending=False).drop_duplicates().head(10)
display(user_preferences.head(10))
reco_title = pred_title(mat_ratings, title_similarity, 3, userId).sort_values(ascending=False).head(10)
print(reco_title)

# 7. Matrice de similarités optimisées (réduites) avec modèle (factorisation de composantes)

## 7.1 SVD (Single Value Decomposition)

In [ ]:
svd = TruncatedSVD(n_components = 12)
title_ratings_reduced = svd.fit_transform(title_ratings)
print(title_ratings_reduced.shape)
title_similarity_reduced = pd.DataFrame(
    dist.cosine_similarity(title_ratings_reduced),
    index=title_mapping, 
    columns=title_mapping
)
display(title_similarity_reduced)
sns.heatmap(title_similarity_reduced.iloc[:30,:30], center=0)

In [ ]:
# Application de la prediction
userId = 7
user_preferences = df_gr[(df_gr['user_id']==userId) & (df_gr['rating']>=4)]
user_preferences.sort_values('rating', ascending=False).drop_duplicates().head(10)
display(user_preferences.head(10))
reco_title = pred_title(mat_ratings, title_similarity_reduced, 3, userId).sort_values(ascending=False).head(10)
print(reco_title)

# 8. Librairie de modèle tout prêts : Surprise